In [1]:
from datasets import Dataset, DatasetDict
from transformers import (
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizerFast,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    pipeline
)
import tokenizers

import os
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

import nlpsig

import random
import math
seed = 2022

## Language dataset

In the `data/` folder, we have several text folders of words from different languages:
- `wordlist_de.txt`: German words
- `wordlist_en.txt`: English words
- `wordlist_fr.txt`: French words
- `wordlist_it.txt`: Italian words
- `wordlist_pl.txt`: Polish words
- `wordlist_sv.txt`: Swedish words

We additionally have a `alphabet.txt` file which just stores the alphabet characters ('a', 'b', 'c', ...).

The task is to split the words into its individual characters and to obtain an embedding for each of them. We can represent a word by a path of its character embeddings and compute its path signature to use as features in predicting the language for which the word belongs.

Here we look at obtaining embeddings using a Transformer model.

In [2]:
ALPHABET_FILE = 'data/alphabet.txt'
with open(ALPHABET_FILE) as f:
    alphabet = f.read().splitlines()
print(alphabet)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Set up Tokenizer for word corpora

If we were to fine-tune an existing pretrained transformer, we could use the same tokenizer that the model was pretrained with. However, here we want to train a model from stratch, and so using a tokenizer that was pretrained on a corpus that looks quite different to ours is suboptimal.

Here, we need to use the `tokenizers` library to set up and train a new tokenizer for our text.

In particular, we're going to start off with a character-based tokenizer (as we're going to split up our words into characters), and train it to our data.

In [3]:
# initialise character based tokenizer
tokenizer = tokenizers.CharBPETokenizer()
tokenizer.train(files=[ALPHABET_FILE], show_progress=False)

# save the tokenizer to "CHAR_BERT/" folder
if not os.path.exists("CHAR_BERT"):
    os.makedirs("CHAR_BERT")

tokenizer.save_model("CHAR_BERT")

['CHAR_BERT/vocab.json', 'CHAR_BERT/merges.txt']

In [37]:
wordlist_files = ["data/wordlist_de.txt",
                  "data/wordlist_en.txt",
                  "data/wordlist_fr.txt",
                  "data/wordlist_it.txt",
                  "data/wordlist_pl.txt",
                  "data/wordlist_sv.txt"]

wordlist_dfs = []
for filename in wordlist_files:
    with open(filename, "r") as f:
        words = f.read().splitlines()
        words_df = pd.DataFrame({"word": words})
        words_df["language"] = filename.split("_")[1][0:2]
        wordlist_dfs.append(words_df)

corpus_df = pd.concat(wordlist_dfs).reset_index(drop=True)

In [38]:
corpus_df

,word,language
0,a,de
1,aal,de
2,aale,de
3,aalen,de
4,aalend,de
...,...,...
3922530,zons,sv
3922531,zoo,sv
3922532,zoologisk,sv
3922533,zoologiska,sv


We can see that there are relatively fewer English words than the other languages...

In [39]:
corpus_df["language"].value_counts()

it    1862929
pl    1517274
fr     198538
de     186027
en      80641
sv      77126
Name: language, dtype: int64

**Question:** what do we do with words that appear twice in the data? i.e. different languages have the same word in their vocabularies, e.g. the word "zoo" is a valid word in english, french, italian, polish and swedish.

In [40]:
corpus_df[corpus_df["word"]=="zoo"]

,word,language
266640,zoo,en
465081,zoo,fr
2326761,zoo,it
3836964,zoo,pl
3922531,zoo,sv


We are going to train our language model on the English words, so taking out a sample of English words from the corpus...

In [48]:
# set seed for sampling
random.seed(seed)

n_words = 60000
english_train = corpus_df[corpus_df["language"]=="en"].sample(n_words)
english_train = english_train.reset_index(drop=True)

# remove the words that we use to train language model from the corpus
corpus_df = corpus_df.drop(english_train.index)
corpus_df = corpus_df.reset_index(drop=True)

In [49]:
english_train

,word,language
0,fishhook,en
1,daffodils,en
2,rickety,en
3,manse,en
4,lightfaced,en
...,...,...
59995,certify,en
59996,shameless,en
59997,nose,en
59998,nymphets,en


To make the dataset bit more manageable, I'll just take a sample of each of the languages.

In [50]:
# set seed for sampling
random.seed(seed)
balanced = True

# take 
n_words = 12000
if balanced:
    languages = corpus_df["language"].unique()
    words_per_language = math.floor(n_words / len(languages))
    corpus_sample_df = pd.concat(
        [corpus_df[corpus_df["language"]==lang].sample(words_per_language, random_state=seed)
         for lang in languages]
    )
    corpus_sample_df = corpus_sample_df.reset_index(drop=True)
else:
    corpus_sample_df = corpus_df.iloc[random.sample(range(len(corpus_df)), n_words)]
    corpus_sample_df = corpus_sample_df.reset_index(drop=True)

corpus_sample_df

,word,language
0,sterilster,de
1,unterschiedenes,de
2,verschuldende,de
3,verbratender,de
4,staatsrechtlich,de
...,...,...
11995,ifatt,sv
11996,formgavs,sv
11997,bestods,sv
11998,kurirers,sv


In [51]:
corpus_sample_df["language"].value_counts()

de    2000
en    2000
fr    2000
it    2000
pl    2000
sv    2000
Name: language, dtype: int64

## Training a language model

We want to train a masked language model for our corpus of English words. In particular, we mask out particular letters and ask our model to try predict the masked letter.

Here, we initialise our tokenizer (here we tokenize by character), data collator (with padding) and set up our transformer model by specifying the config (we use the RoBERTa here).

In [52]:
max_length = 512

# load in tokenizer for architecture
tokenizer = RobertaTokenizerFast.from_pretrained('CHAR_BERT/',
                                                 max_len=max_length)

# set up data_collator to use (intially just one that adds padding)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# initialise transformer architecture (random weights)
config_args = {"vocab_size": tokenizer.backend_tokenizer.get_vocab_size(),
               "hidden_size": 768,
               "max_length": 512,
               "max_position_embeddings": max_length + 2,
               "hidden_dropout_prob": 0.1,
               "num_attention_heads": 12,
               "num_hidden_layers": 6,
               "type_vocab_size": 1}

config = RobertaConfig(**config_args)
model = RobertaForMaskedLM(config=config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Using the `TextEncoder` class

The `TextEncoder` class in the `nlpsig` package is able to take a dataframe with a column of text. We can use this class to obtain embeddings for the input text, or to train the model with the input text.

Note: In the initial writing of the package, the idea was that we can fit our transformer to some data (some text), and then we can obtain embeddings for them. But in our setting, we actually want to fit our data to our sample of English words (which we call `english_train`), but then obtain embeddings for our sample of the remaining words (which we call `corpus_sample_df`) - noting that this also contains some English words.

So we will actually use two instances of `TextEncoder` - one to pass in `corpus_df` and train the model. And then another to obtain embeddings for the words in `corpus_sample_df`. This is not optimal and not clean, but some adjustment to `TextEncoder` will be able. In particular, we can perhaps make changes to the `.tokenize_text()` method (which we will see how to use later) which can take in some external text data. 

I envisage that we can pass in different data to train the model rather than training it on the data that is passed...

But for now...

In [53]:
english_train.head()

,word,language
0,fishhook,en
1,daffodils,en
2,rickety,en
3,manse,en
4,lightfaced,en


In [54]:
text_encoder = nlpsig.TextEncoder(df=english_train,
                                  feature_name="word",
                                  model=model,
                                  config=config,
                                  tokenizer=tokenizer,
                                  data_collator=data_collator)

We can tokenize the text with the `.tokenize_text()` method, which tokenizes each of the items in the column of the dataframe that we passed in. So in the above, we tokenise the `word` column of the `english_train` dataframe.

In [55]:
text_encoder.tokenize_text()

[INFO] Setting return_special_tokens_mask=True
[INFO] Tokenizing the datatset...


  0%|          | 0/60 [00:00<?, ?ba/s]

[INFO] Saving the tokenized text for each sentence into `.df['tokens']`...


  0%|          | 0/60 [00:00<?, ?ba/s]

[INFO] Creating tokenized dataframe and setting in `.tokenized_df` attribute...
[INFO] Note: 'text_id' is the column name for denoting the corresponding text id


Dataset({
    features: ['word', 'language', 'input_ids', 'attention_mask', 'special_tokens_mask', 'tokens'],
    num_rows: 60000
})

Note that the `text_encoder` object (instance of `TextEncoder`) also keeps the data as a Huggingface Dataset object too which is stored in the `.dataset` attribute of the object:

In [56]:
text_encoder.dataset

Dataset({
    features: ['word', 'language', 'input_ids', 'attention_mask', 'special_tokens_mask', 'tokens'],
    num_rows: 60000
})

We can see that we have tokenized this as there are `input_ids`, `attention_mask`, `special_tokens_mask`, and `tokens` features in the dataset.

In [57]:
text_encoder.dataset["word"][0]

'fishhook'

In [58]:
text_encoder.dataset["input_ids"][0]

[53, 6, 9, 19, 8, 8, 15, 15, 11, 54]

We can see that this word has been tokenized by character:

In [59]:
text_encoder.dataset["tokens"][0]

['f', 'i', 's', 'h', 'h', 'o', 'o', 'k']

We can also see that we have saved the tokenized text in the `'token'` column of the dataframe stored in `.df`:

In [60]:
text_encoder.df

,word,language,tokens
0,fishhook,en,"[f, i, s, h, h, o, o, k]"
1,daffodils,en,"[d, a, f, f, o, d, i, l, s]"
2,rickety,en,"[r, i, c, k, e, t, y]"
3,manse,en,"[m, a, n, s, e]"
4,lightfaced,en,"[l, i, g, h, t, f, a, c, e, d]"
...,...,...,...
59995,certify,en,"[c, e, r, t, i, f, y]"
59996,shameless,en,"[s, h, a, m, e, l, e, s, s]"
59997,nose,en,"[n, o, s, e]"
59998,nymphets,en,"[n, y, m, p, h, e, t, s]"


We also store the tokens in `.tokens` attribute.

In [64]:
text_encoder.tokens

Dataset({
    features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 60000
})

After applying the `.tokenize_text()` method, we store a tokenized dataframe in the `.tokenized_df` attribue. Here, we have each token in our corpus and their associated `'text_id'` (which is just the index they were given in the original dataframe that we pass):

In [61]:
text_encoder.tokenized_df

,text_id,language,tokens
0,0,en,f
1,0,en,i
2,0,en,s
3,0,en,h
4,0,en,h
...,...,...,...
511576,59999,en,e
511577,59999,en,l
511578,59999,en,l
511579,59999,en,e


So if we looked at `text_id==0`:

In [62]:
text_encoder.tokenized_df[text_encoder.tokenized_df["text_id"]==0]

,text_id,language,tokens
0,0,en,f
1,0,en,i
2,0,en,s
3,0,en,h
4,0,en,h
5,0,en,o
6,0,en,o
7,0,en,k


If we had passed in a pre-trained model (remember above, we just initialised one with a config and so have random weight), we can obtain token embeddings by the `.obtain_embeddings()` method. 

There are many ways in which one can get embeddings from the transformer network, as the output is the layers for the full network. A few ways are:

- Returning the output of a particular hidden layer
    - use `.obtain_embeddings(method = "hidden_layer", layers = l)` where `l` is the layer you want
    - If no layer is requested, it will just give you the second-to-last hidden layer of the transformer network.
- Concatenate the output of several hidden layers
    - use `.obtain_embeddings(method = "concatenate", layers = [l_1, l_2, ...])` where `[l_1, l_2, ...]` is a list of layers you want to concatenate
- Element-wise sum the output of several hidden layers
    - use `.obtain_embeddings(method = "sum" , layers = [l_1, l_2, ...])` where `[l_1, l_2, ...]` is a list of layers you want to sum
- Mean the output of several hidden layers
    - use `.obtain_embeddings(method = "mean" , layers = [l_1, l_2, ...])` where `[l_1, l_2, ...]` is a list of layers you want to mean

If a more custom way to obtain embeddings from the hidden layers, you can specify what layers you want, and it will return them (i.e. using `.obtain_embeddings(method = "hidden_layer", layers = [l_1, l_2, ...])` where `[l_1, l_2, ...]` is a list of hidden layers you want) and so the output will be a 3-dimensional array with dimensions `[layer, token, embedding]` for which you would need to combine in such a way that you would have an embedding for each token. The above methods would return a 2-dimensional array with dimensions `[token, embedding]`.

In the below, we just obtain the second-to-last hidden layer of the network:

In [63]:
token_embeddings = text_encoder.obtain_embeddings(method = "hidden_layer")

100%|██████████████████████████████| 600/600 [03:02<00:00,  3.29it/s]


By inspecting the shape of this, we can see that we have a 2-dimensional array with dimensions `[token, embedding]` where the embeddings are 768 dimensional in this network.

In [65]:
token_embeddings.shape

(511581, 768)

Now that we have token embeddings for each text, it is possible to pool these embeddings to obtain an embedding for the full text (for this case, this embedding would represent the word itself. We can use the `.pool_token_embeddings()` method for doing this.

Again, there are several methods and full details can be found in the documentation, but a few are:

- take the mean of the token embeddings
    - use `.pool_token_embeddings(method = "mean")`
- take the element-wise max of the token embeddings
    - use `.pool_token_embeddings(method = "max")`
- take the element-wise sum of the token embeddings
    - use `.pool_token_embeddings(method = "sum")`
- take the token-embedding for the CLS token
    - this is a special token that is used in some transformers like BERT
    - but this is only available to us if we set `skip_special_tokens=False` when tokenizing the text (note by default, this is set to `True`)
    - use `.pool_token_embeddings(method = "sum")`
        - this will produce an error if the CLS token is not available...

In [66]:
pooled_embeddings = text_encoder.pool_token_embeddings()

100%|████████████████████████| 60000/60000 [00:23<00:00, 2530.65it/s]


Again, we can inspect the shape and we can see that we have embeddings for each of our words:

In [67]:
pooled_embeddings.shape

(60000, 768)

## Training the model

The above embeddings will not be good for any downstream task as the model itself has not been trained to the text. For this we will use other methods in the `TextEncoder` class which allows us to do this by using the Huggingface trainer API.

First, we need to set up a data collator for training our model.

In [69]:
# set up data_collator for language modelling (has dynamic padding)
data_collator_for_LM = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                       mlm=True,
                                                       mlm_probability=0.15)

To train our dataset, we will split it into a train, validation and test set with the `.split_dataset()` method.

We can set up the trainer's arguments with `.set_up_training_args()` which sets up a `TrainingArguments` object (from the `transformers` package) and stores it in the `.training_args` attribute. And lastly, we set up a `Trainer` object (from the `transformers` package) and store it in the `.trainer` attribute.

In [70]:
text_encoder.split_dataset()
text_encoder.set_up_training_args(output_dir="CHAR_BERT_trained",
                                  num_train_epochs=20,
                                  per_device_train_batch_size=128,
                                  seed=seed)
text_encoder.set_up_trainer(data_collator=data_collator_for_LM)

[INFO] Splitting up dataset into train / validation / test sets, and saving to `.dataset_split`.
[INFO] Setting up TrainingArguments object and saving to `.training_args`.
[INFO] Setting up Trainer object, and saving to `.trainer`.


Once everything is set up, we just train our model by calling `.fit_transformer_with_trainer_api()` method.

In [71]:
text_encoder.fit_transformer_with_trainer_api()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: word, special_tokens_mask, tokens, language. If word, special_tokens_mask, tokens, language are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
/Users/rchan/opt/miniconda3/envs/py38esig/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 48000
  Num Epochs = 20
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 7500
  Number of trainable parameters = 43560249


[INFO] Training model with 43560249 parameters...


Epoch,Training Loss,Validation Loss
1,No log,2.163147
2,2.399900,1.925708
3,1.965900,1.826439
4,1.847300,1.722984
5,1.847300,1.702801
6,1.761900,1.667223
7,1.699100,1.640108
8,1.669400,1.608557
9,1.669400,1.577421
10,1.627500,1.583390


The following columns in the evaluation set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: word, special_tokens_mask, tokens, language. If word, special_tokens_mask, tokens, language are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8
Saving model checkpoint to CHAR_BERT_trained/checkpoint-500
Configuration saved in CHAR_BERT_trained/checkpoint-500/config.json
Model weights saved in CHAR_BERT_trained/checkpoint-500/pytorch_model.bin
tokenizer config file saved in CHAR_BERT_trained/checkpoint-500/tokenizer_config.json
Special tokens file saved in CHAR_BERT_trained/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: word, special_tokens_mask, tokens, language. If word, special_tokens_mask, tokens, language are not expecte

***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8
Saving model checkpoint to CHAR_BERT_trained/checkpoint-5000
Configuration saved in CHAR_BERT_trained/checkpoint-5000/config.json
Model weights saved in CHAR_BERT_trained/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in CHAR_BERT_trained/checkpoint-5000/tokenizer_config.json
Special tokens file saved in CHAR_BERT_trained/checkpoint-5000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: word, special_tokens_mask, tokens, language. If word, special_tokens_mask, tokens, language are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8
Saving model checkpoint to CHAR_BERT_trained/checkpoint-5500
Configuration saved in CHAR_BERT_trained/checkpoint-5500/config.json
Model weights saved in CHAR_BERT_trai

[INFO] Training completed!


Saving our model:

In [75]:
text_encoder.trainer.save_model("CHAR_BERT_trained/")

Saving model checkpoint to CHAR_BERT_trained/
Configuration saved in CHAR_BERT_trained/config.json
Model weights saved in CHAR_BERT_trained/pytorch_model.bin
tokenizer config file saved in CHAR_BERT_trained/tokenizer_config.json
Special tokens file saved in CHAR_BERT_trained/special_tokens_map.json


## Evaluating trained model

Evaluating the performance on predicting the masked letter for the test dataset. To do this, for each word in our test dataset, we will mask each letter on its own and ask the model to predict the masked letter. So for a 5 letter word, we have 5 predictions to make - one for each letter given the other letters.

For our tokenizer, we see that `\<mask>` is used as the mask token:

In [76]:
text_encoder.tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

In [77]:
def compute_masked_character_accuracy(fill_mask, words):
    was_correct = []
    print(f"Evaluating with {len(words)} words")
    for word in tqdm(words):
        masked_strings = [word[:i] + '<mask>' + word[i+1:] for i in range(len(word))]
        predictions = [fill_mask(word)[0]['sequence'] for word in masked_strings]
        was_correct += [pred == word for pred in predictions]
    
    acc = np.sum(was_correct) / len(was_correct)
    print(f"Accuracy: {acc}")
    return acc

fill_mask = pipeline("fill-mask",
                     model="CHAR_BERT_trained",
                     tokenizer="CHAR_BERT_trained")

compute_masked_character_accuracy(fill_mask, 
                                  text_encoder.dataset_split["test"]["word"])

loading configuration file CHAR_BERT_trained/config.json
Model config RobertaConfig {
  "_name_or_path": "CHAR_BERT_trained",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_length": 512,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 57
}

loading configuration file CHAR_BERT_trained/config.json
Model config RobertaConfig {
  "_name_or_path": "CHAR_BERT_trained",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bo

Evaluating with 6000 words


100%|████████████████████████████| 6000/6000 [23:59<00:00,  4.17it/s]

Accuracy: 0.5537810994198052


0.5537810994198052

## Obtaining a path for each word

Now that we have trained our model, we want to obtain embeddings for the words in `corpus_df`. Currently, `TextEncoder` only works with the data that is passed into the function and stored in `.df` and `.dataset`, so we need to initialise a new `TextEncoder` object with the `corpus_df` dataframe and also the trained model.

We can then obtain embeddings easily (recall from above we first need to tokenize the text, and then use the `.obtain_embeddings()` and `.pool_token_embeddings()` methods to do this).

In [81]:
text_encoder_2 = nlpsig.TextEncoder(df=corpus_sample_df,
                                    feature_name="word",
                                    model=text_encoder.model,
                                    config=text_encoder.config,
                                    tokenizer=text_encoder.tokenizer,
                                    data_collator=text_encoder.data_collator)

In [82]:
text_encoder_2.tokenize_text()
token_embeddings = text_encoder_2.obtain_embeddings(method = "hidden_layer")
pooled_embeddings = text_encoder_2.pool_token_embeddings()

[INFO] Setting return_special_tokens_mask=True
[INFO] Tokenizing the datatset...


  0%|          | 0/12 [00:00<?, ?ba/s]

[INFO] Saving the tokenized text for each sentence into `.df['tokens']`...


  0%|          | 0/12 [00:00<?, ?ba/s]

[INFO] Creating tokenized dataframe and setting in `.tokenized_df` attribute...
[INFO] Note: 'text_id' is the column name for denoting the corresponding text id



 92%|███████████████████████████▊  | 111/120 [00:29<00:02,  3.95it/s]


100%|██████████████████████████████| 120/120 [00:32<00:00,  3.75it/s]

100%|████████████████████████| 12000/12000 [00:01<00:00, 7098.62it/s]


In [83]:
text_encoder_2.tokenized_df

,text_id,language,tokens
0,0,de,s
1,0,de,t
2,0,de,e
3,0,de,r
4,0,de,i
...,...,...,...
130063,11999,sv,n
130064,11999,sv,g
130065,11999,sv,a
130066,11999,sv,r


In [84]:
token_embeddings.shape

(130068, 768)

In [85]:
pooled_embeddings.shape

(12000, 768)

## Dimension reduction

We can perform dimension reduction with `nlpsig` using the `DimReduce` class. Here, we will use UMAP (implemented using the [`umap-learn`](https://umap-learn.readthedocs.io/en/latest/api.html) package, but there are other standard methods available:
- PCA (implemented using [`scikit-learn`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html))
    - `method="pca"`
- TSNE (implemented using [`scikit-learn`](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html))
    - `method="tsne"`
- Post Processing Algorithm (PPA) with PCA (PPA-PCA)
    - `method="ppapca"`
    - see _Mu, J., Bhat, S., and Viswanath, P. (2017). All-but-the-top: Simple and effective postprocessing for word representations. arXiv preprint arXiv:1702.01417._
- PPA-PCA-PPA
    - `method="ppapacppa"`
    - see _Raunak, V., Gupta, V., and Metze, F. (2019). Effective dimensionality reduction for word embeddings. In Proceedings of the 4th Workshop on Representation Learning for NLP (RepL4NLP- 2019), pages 235–243._

In [92]:
reduction = nlpsig.DimReduce(method="umap",
                             n_components=10,
                             dim_reduction_kwargs={
                                 "metric": "cosine",
                             })
embeddings_reduced = reduction.fit_transform(token_embeddings)

In [93]:
embeddings_reduced.shape

(130068, 10)

As we have embeddings for each token, we can obtain a path for each word by constructing a path of the token embeddings. To do this, we can use the `PrepareData` class and pass in our tokenized dataframe (the dataframe where we have each token in our data and we also have the corresponding id for each word which is saved in the `text_id` column of the tokenized dataframe.

We pass in the column which defines the ids, `text_id`, the column which defines the labels, `language`, the token embeddings and the pooled embeddings.

In [95]:
dataset = nlpsig.PrepareData(text_encoder_2.tokenized_df,
                             id_column="text_id",
                             labels_column="language",
                             embeddings=token_embeddings,
                             embeddings_reduced=embeddings_reduced)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Note 'datetime' is not a column in `.df`, so only 'timeline_index' is added.
[INFO] As 'datetime' is not a column in `.df`, we assume that the data is ordered by time with respect to the id.
[INFO] Adding 'timeline_index' feature...


We can construct a path by using the `.pad()` method, and result of this is a multi-dimensional array or tensor (in particular a numpy array or PyTorch tensor) which can be then used in some downstream task. It is called "pad" because arrays and tensors are rectangular and if there are cases where there isn't enough data (e.g. if a word only has 3 letters/tokens and we want to make paths of length 4), we "pad" with either the last token embedding (set `zero_padding=False`) or with zeros (set `zero_padding=True`).

Here, we construct paths by setting a length of the paths (we call this method `k_last` in the code and we have to specify the length with `k=10`). We alternatively can construct to the longest word possible (by setting `method="max"`). The `time_feature` argument allows us to specify what time features we want to keep. Here we don't have any besides the index in which the word is, which is given by `timeline_index` and we choose not to standardise that by specifying `standardise_time_feature=False`.

In [96]:
word_path = dataset.pad(pad_by="id",
                        zero_padding=True,
                        method="k_last",
                        k=10,
                        time_feature=["timeline_index"],
                        standardise_time_feature=False,
                        embeddings="dim_reduced")

[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|████████████████████████| 12000/12000 [00:06<00:00, 1888.12it/s]


By inspecting the shape of `word_path`, we see that we have a path for each word and the dimension of the array is `[batch, length of path, channels]`.

In [97]:
word_path.shape

(12000, 10, 13)

In [98]:
len(dataset.df["text_id"].unique())

12000

We store this array as a dataframe in `.df_padded` so that you can see what the columns correspond to, where columns beginning with `e` denote the dimensions of embeddings obtained from the transformer.

In [99]:
dataset.df_padded

,timeline_index,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,text_id,language
0,0,9.316443,4.845233,4.327503,12.503558,6.330883,9.639407,5.759614,5.490386,8.510323,4.942838,0,de
1,1,8.887857,5.960692,6.048943,7.624972,4.420071,1.446805,5.870730,4.063160,2.149124,3.452542,0,de
2,2,9.593655,5.001677,4.819080,0.763647,8.600629,6.855587,0.544169,2.737365,8.712207,4.943607,0,de
3,3,10.738387,5.116134,2.100512,4.531888,4.817560,-0.247809,8.997357,2.103364,9.245868,4.446329,0,de
4,4,9.959823,4.621105,7.982635,2.020440,7.114519,8.217345,5.923898,9.328344,4.378319,5.210546,0,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,6,10.747741,4.271384,1.787655,3.467277,0.551456,8.785113,2.832803,2.179157,4.572252,7.398472,11999,sv
119996,7,9.875523,5.655285,4.204875,3.621913,5.814768,-0.771930,3.293646,4.322257,3.750905,4.650285,11999,sv
119997,8,18.076136,4.470689,4.693927,5.279918,4.913960,5.447477,3.608374,5.227146,7.097644,5.407924,11999,sv
119998,9,9.784361,5.126089,2.388112,4.863431,5.934732,0.836809,7.378283,1.178062,8.950499,3.723052,11999,sv


We can see that the first column corresponds to the index, the columns beginning with `d` correspond to the dimension-reduced embeddings (which were 10 dimensional), and we also have the corresponding text-id and language (which we passed in the label above). If we look at the first word:

In [113]:
# still has the labels and the ids
dataset.df_padded[dataset.df_padded["text_id"]==0]

,timeline_index,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,text_id,language
0,0,9.316443,4.845233,4.327503,12.503558,6.330883,9.639407,5.759614,5.490386,8.510323,4.942838,0,de
1,1,8.887857,5.960692,6.048943,7.624972,4.420071,1.446805,5.870730,4.063160,2.149124,3.452542,0,de
2,2,9.593655,5.001677,4.819080,0.763647,8.600629,6.855587,0.544169,2.737365,8.712207,4.943607,0,de
3,3,10.738387,5.116134,2.100512,4.531888,4.817560,-0.247809,8.997357,2.103364,9.245868,4.446329,0,de
4,4,9.959823,4.621105,7.982635,2.020440,7.114519,8.217345,5.923898,9.328344,4.378319,5.210546,0,de
5,5,10.099976,5.472713,4.311344,0.887817,1.529893,2.656477,5.109613,1.446850,1.655324,6.259146,0,de
6,6,8.957418,4.832905,7.315309,11.573713,6.240843,5.481841,0.409815,3.607513,7.758046,5.539202,0,de
7,7,9.213166,6.091189,6.323124,7.939425,4.316284,0.784012,6.178899,4.419389,1.761438,3.250692,0,de
8,8,9.141431,5.303648,4.492472,1.272192,8.932236,6.521078,0.421085,2.273735,8.782001,5.859922,0,de
9,9,9.047493,4.900075,5.746224,3.393111,4.659221,0.555108,9.175676,2.258206,10.495255,6.689592,0,de


In [124]:
text_encoder_2.df.iloc[0]

word                            sterilster
language                                de
tokens      [s, t, e, r, i, l, s, t, e, r]
Name: 0, dtype: object

We pick out a word which has less than 10 letters, and we can see that the path is padded with zeros and we give these a label `-1` to denote that they have been added.

Note that for padding, the method pads from below by default, but we can pad by above by setting `pad_from_below=False`.

In [127]:
dataset.df_padded[dataset.df_padded["text_id"]==3320]

,timeline_index,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,text_id,language
33200,0,9.611249,4.867899,0.527855,6.496469,4.894460,7.605602,8.416375,8.800124,8.227205,2.436413,3320,en
33201,1,10.523488,5.070083,4.103313,2.341068,7.382311,3.989184,8.398027,8.477536,-0.798858,6.452246,3320,en
33202,2,6.772434,4.345559,3.611884,6.029821,3.352861,4.986651,4.431797,4.751046,5.456618,2.363499,3320,en
33203,3,9.021976,5.100981,1.545537,3.304732,-0.104942,8.160237,3.559365,2.423001,4.994720,6.719332,3320,en
33204,4,11.743196,5.022210,4.634274,3.535622,7.453012,4.446137,8.867769,8.490068,-0.583250,6.533859,3320,en
33205,5,10.112203,3.670740,3.416387,5.942775,-0.490579,3.622755,5.881639,5.320231,5.691600,4.794139,3320,en
33206,6,11.029837,4.917490,5.932513,0.115193,6.846435,7.232195,-0.030283,2.961208,8.966331,4.555634,3320,en
33207,7,9.234504,5.119550,4.571543,2.619485,-3.523274,4.053810,1.409377,7.513707,6.222019,4.357717,3320,en
33208,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3320,-1
33209,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3320,-1


In [128]:
text_encoder_2.df.iloc[3320]

word                        cognomen
language                          en
tokens      [c, o, g, n, o, m, e, n]
Name: 3320, dtype: object

For words which are longer than 10 letters, we only take the last 10:

In [129]:
dataset.df_padded[dataset.df_padded["text_id"]==10]

,timeline_index,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,text_id,language
100,4,6.765463,5.742875,7.315415,4.297583,7.099874,6.667671,1.551378,6.853375,2.757002,4.997705,10,de
101,5,10.138185,4.709245,2.165445,3.375592,0.713781,8.835148,2.635751,2.438843,4.611350,7.229488,10,de
102,6,6.249787,3.974104,4.045012,5.929241,3.161420,4.917922,4.442468,4.858975,5.672881,2.111718,10,de
103,7,9.160442,4.917056,7.164062,11.719777,6.225347,5.443470,0.693212,3.893485,7.622388,5.271125,10,de
104,8,8.451878,4.867537,3.390759,8.272752,5.518309,6.608521,5.366010,5.223764,3.831670,2.555008,10,de
105,9,11.370544,4.847795,4.392432,3.113414,7.140684,4.390954,9.180496,8.712993,-0.518369,6.244632,10,de
106,10,9.566058,5.396273,4.441339,0.899723,1.663759,2.592971,5.305820,1.212458,1.705024,6.345337,10,de
107,11,9.422722,5.814419,4.103125,1.476479,2.338355,2.783099,5.836319,0.453386,1.190403,5.976224,10,de
108,12,9.085876,5.155453,4.886012,0.073813,7.100619,6.242529,-0.049043,2.893509,9.982448,4.248672,10,de
109,13,9.709391,4.826661,5.155285,13.915672,4.762872,3.235626,2.735438,4.633454,7.020216,6.252982,10,de


In [130]:
text_encoder_2.df.iloc[10]

word                                    salbungsvolles
language                                            de
tokens      [s, a, l, b, u, n, g, s, v, o, l, l, e, s]
Name: 10, dtype: object

To obtain a path as a torch tensor, we use the `.get_torch_path()` method which by default keeps the time features and will remove the id and label columns.

In [131]:
# by default keeps the time features
torch_word_path = dataset.get_torch_path()
torch_word_path.shape

torch.Size([12000, 10, 11])

In [132]:
torch_word_path[0]

tensor([[ 0.0000,  9.3164,  4.8452,  4.3275, 12.5036,  6.3309,  9.6394,  5.7596,
          5.4904,  8.5103,  4.9428],
        [ 1.0000,  8.8879,  5.9607,  6.0489,  7.6250,  4.4201,  1.4468,  5.8707,
          4.0632,  2.1491,  3.4525],
        [ 2.0000,  9.5937,  5.0017,  4.8191,  0.7636,  8.6006,  6.8556,  0.5442,
          2.7374,  8.7122,  4.9436],
        [ 3.0000, 10.7384,  5.1161,  2.1005,  4.5319,  4.8176, -0.2478,  8.9974,
          2.1034,  9.2459,  4.4463],
        [ 4.0000,  9.9598,  4.6211,  7.9826,  2.0204,  7.1145,  8.2173,  5.9239,
          9.3283,  4.3783,  5.2105],
        [ 5.0000, 10.1000,  5.4727,  4.3113,  0.8878,  1.5299,  2.6565,  5.1096,
          1.4469,  1.6553,  6.2591],
        [ 6.0000,  8.9574,  4.8329,  7.3153, 11.5737,  6.2408,  5.4818,  0.4098,
          3.6075,  7.7580,  5.5392],
        [ 7.0000,  9.2132,  6.0912,  6.3231,  7.9394,  4.3163,  0.7840,  6.1789,
          4.4194,  1.7614,  3.2507],
        [ 8.0000,  9.1414,  5.3036,  4.4925,  1.2722,  8

We can choose to ignore the time features by setting `include_time_features=False`:

In [133]:
# ingore time features (ignore first column of timeline_index)
torch_word_path = dataset.get_torch_path(include_time_features=False)
torch_word_path.shape

torch.Size([12000, 10, 10])

In [134]:
torch_word_path[0]

tensor([[ 9.3164,  4.8452,  4.3275, 12.5036,  6.3309,  9.6394,  5.7596,  5.4904,
          8.5103,  4.9428],
        [ 8.8879,  5.9607,  6.0489,  7.6250,  4.4201,  1.4468,  5.8707,  4.0632,
          2.1491,  3.4525],
        [ 9.5937,  5.0017,  4.8191,  0.7636,  8.6006,  6.8556,  0.5442,  2.7374,
          8.7122,  4.9436],
        [10.7384,  5.1161,  2.1005,  4.5319,  4.8176, -0.2478,  8.9974,  2.1034,
          9.2459,  4.4463],
        [ 9.9598,  4.6211,  7.9826,  2.0204,  7.1145,  8.2173,  5.9239,  9.3283,
          4.3783,  5.2105],
        [10.1000,  5.4727,  4.3113,  0.8878,  1.5299,  2.6565,  5.1096,  1.4469,
          1.6553,  6.2591],
        [ 8.9574,  4.8329,  7.3153, 11.5737,  6.2408,  5.4818,  0.4098,  3.6075,
          7.7580,  5.5392],
        [ 9.2132,  6.0912,  6.3231,  7.9394,  4.3163,  0.7840,  6.1789,  4.4194,
          1.7614,  3.2507],
        [ 9.1414,  5.3036,  4.4925,  1.2722,  8.9322,  6.5211,  0.4211,  2.2737,
          8.7820,  5.8599],
        [ 9.0475,  